In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from itertools import islice
from urllib.parse import urlparse
import ast


In [2]:
# Load filtered an streamed FineWeb 2 dataset
dataset = load_dataset("HuggingFaceFW/fineweb-2", split="train", data_dir="data/dan_Latn", streaming=True)

In [19]:
def get_batches(ds, batch_size):
    iterator = iter(ds)
    while True:
        batch = list(islice(iterator, batch_size))
        if not batch:
            break
        yield batch


def domain_distribution(df):
    if "domain" not in df.columns:
        raise ValueError("DataFrame does not contain a 'domain' column.")
    
    distribution = df["domain"].value_counts().sort_values(ascending=False)
    return distribution

import pandas as pd
import ast

def save_text_label(id, text, label, path="fineweb2_data.csv"):
    try:
        df = pd.read_csv(path)
        # Ensure 'id' column exists
        if 'id' not in df.columns:
            df['id'] = None
        df["label"] = df["label"].apply(ast.literal_eval)  # Convert label column from str to list
    except FileNotFoundError:
        df = pd.DataFrame(columns=["text", "label", "id"])

    # Find matching text (original behavior)
    match = df[df["text"] == text]

    if not match.empty:
        idx = match.index[0]
        labels = df.at[idx, "label"]
        if label not in labels:
            labels.append(label)
            df.at[idx, "label"] = labels
        # Existing rows: Keep original ID (DO NOT update with new id)
    else:
        # New rows: Add with provided id
        new_row = pd.DataFrame([{"text": text, "label": [label], "id": id}])
        df = pd.concat([df, new_row], ignore_index=True)

    df.to_csv(path, index=False)

In [20]:
Batch_Size = 1000
Batch_Iterations = 13
Remove_Domains = ["www.texaspoker.dk"]

batch_generator = get_batches(dataset, batch_size=Batch_Size)

# Initialize storage
all_records = pd.DataFrame()
all_domains = []

# Process in batches
for i in range(Batch_Iterations):
    print(f"Batch {i+1} of {Batch_Iterations}")
    batch = next(batch_generator)

    records = [{
        "text": example["text"],
        "minhash_cluster_size": example.get("minhash_cluster_size"),
        "url": example.get("url"),
        "id": example.get("id")
    } for example in batch]

    temp_Records = pd.DataFrame(records)
    temp_Records["domain"] = temp_Records["url"].apply(lambda url: urlparse(url).netloc if url else None)

    # Filter out domains
    filtered_Records = temp_Records[~temp_Records["domain"].isin(Remove_Domains)].reset_index(drop=True)

    all_records = pd.concat([all_records, filtered_Records], ignore_index=True)
    all_domains.extend(filtered_Records["domain"].unique())



    

Batch 1 of 13
Batch 2 of 13
Batch 3 of 13
Batch 4 of 13
Batch 5 of 13
Batch 6 of 13
Batch 7 of 13
Batch 8 of 13
Batch 9 of 13
Batch 10 of 13
Batch 11 of 13
Batch 12 of 13
Batch 13 of 13


In [11]:
print(domain_distribution(all_records))
print("Total Domains:",len(all_domains))

domain
da.wikipedia.org                                                       929
www.andersen.sdu.dk                                                    366
ekstrabladet.dk                                                        311
www.bt.dk                                                              292
www.agoda.com                                                          278
www.tripadvisor.dk                                                     200
www.jetcost.dk                                                         174
www.dagens.dk                                                          165
deborah.dk                                                             158
www.denstoredanske.dk                                                  149
europa.eu                                                              115
www.wimdu.dk                                                           110
www.avisen.dk                                                          110
www.trustpilot.co.

In [7]:
def browse_samples_by_domain(all_domains, all_records):
    pd.set_option('display.max_colwidth', 100)  # limit text preview
    pd.set_option('display.max_rows', False)    # show all rows if needed

    for domain in all_domains:
        domain_samples = all_records[all_records["domain"] == domain]

        print(f"\n--- Domain: {domain} ---")
        if domain_samples.empty:
            print("No samples available.")
        else:
            sample = domain_samples.sample(n=1).iloc[0]
            display(sample)
            print(f"Sample text:\n{sample['text']}\n")

        user_input = input("Press ENTER to continue or 'q' to quit: ").strip().lower()
        if user_input == 'q':
            print("Exiting...")
            break

In [ ]:
browse_samples_by_domain(all_domains, all_records)

In [ ]:
# 6, good, 31, good, 183, good, 1490 good, 2509, good, 164, good kinda, 198, good, 

In [140]:
check_domain = "denstoredanske.dk"

# Show all rows
pd.set_option('display.max_rows', None)

# Limit text column width (e.g. to 100 characters)
pd.set_option('display.max_colwidth', 500)

# Filter and display
all_records[all_records["domain"] == check_domain]

,text,minhash_cluster_size,url,id,domain
7739,"Den amerikanske sanger og skuespiller Whitney Houston døde, 48 år gammel. Hun brød med sin kraftige stemme igennem med albummet Whitney Houston i 1985 og havde siden bl.a. succes i filmen The Bodyguard.\nDen amerikanske politiker Sarah Louise Heath Palin blev født. Hun blev 2006 den første kvindelige guvernør i Alaska og var vicepræsidentkandidat for Det Republikanske Parti i 2008, hvor hun sammen med John McCain stillede op mod Barack Obama og Joe Biden. Sarah Palin har flere gange fremståe...",134,http://denstoredanske.dk/index.php?title=Sprog%2C_religion_og_filosofi/Sprog/Fremmedord/rh-r%C3%A5/rima,<urn:uuid:3fdc2781-ca52-49b9-9ece-ea8704489249>,denstoredanske.dk
7798,"Den amerikanske dramatiker Arthur Miller døde, 89 år gammel. Han var en af 1900-tallets største amerikanske teaterpersonligheder. Hans berømte En sælgers død skildrer den amerikanske succesdrøm som hul og overfladisk.\nDen russiske filminstruktør Sergej Mikhajlovitj Eisenstein døde, 50 år gammel. Eisenstein er verdensberømt for sine montagefilm. Han nåede også — dog uden større succes — en enkelt gang til Hollywood.\nSverige blev kastet ud i en parlamentarisk krise. Statsminister Karl Staaf ...",209,http://denstoredanske.dk/index.php?title=Samfund%2C_jura_og_politik/Samfund/Samfund_og_statistik/penge-_og_kapitalmarked,<urn:uuid:3c086c8d-2c68-46ce-8c8b-03093ee2dd4e>,denstoredanske.dk
7803,"Hans Tausen, 1494-11.11.1561, dansk reformator og biskop, født i Birkende på Fyn. Som ung blev han optaget i johanniterklostret i Antvorskov og studerede fra 1516 i Rostock; senest 1520 var han præsteviet. I 1521-22 studerede han i København, 1522-23 i Leuven og 1523-24 i Wittenberg. Efter at være hjemkaldt gav han i sine prædikener udtryk for lutherske synspunkter, og efter at være overflyttet til Viborg prædikede han under stor tilstrømning renlivet lutherdom. Hans forkyndelse fik da et så...",42,"http://denstoredanske.dk/Sprog,_religion_og_filosofi/Religion_og_mystik/Danske_folkekirke/Hans_Tausen",<urn:uuid:1a559f8c-aa22-40b9-af85-ee0d4e9b11c6>,denstoredanske.dk
7897,"Sudbury, by i Ontario, Canada, ved Trans-Canada Highway 330 km nord for Toronto; 155.200 indb. (2001). Byens historie er knyttet til minedriften i det nærliggende Sudbury Basin, som indledtes få år efter anlæggelsen af jernbanen Canadian Pacific i 1883 og kulminerede i 1960'erne, hvorefter en kombination af vigende efterspørgsel og problemer med en omfattende luftforurening dæmpede aktiviteterne. I takt hermed er byen blevet mindre afhængig af mineselskaberne INCO og Falconbridge, der endnu ...",34,http://denstoredanske.dk/Geografi_og_historie/Canada/Canadas_geografi/Sudbury,<urn:uuid:b07201a7-8a03-4e9c-a0b7-45f37ca8c0a3>,denstoredanske.dk
7911,"lighed, i politisk tænkning det samme som alle menneskers ligeværd.\nEuropas historie viser, at der gennem tiderne er blevet udfoldet bestræbelser for at mindske, hvad der opfattedes som uretfærdige forskelle i status og magt i politisk, social og økonomisk henseende.\nLighed har sammen med frihed og retfærdighed været et centralt begreb i den europæiske politiske og moralfilosofiske tænkning og en fundamental drivkraft i den politiske udvikling.\nLighedsbegrebet har udviklet sig ud fra en p...",82,http://denstoredanske.dk/Samfund%2C_jura_og_politik/Samfund/Statsl%C3%A6re_og_politisk_videnskabsteori/lighed,<urn:uuid:0a964a3f-2d08-45a9-9332-e002f00b005d>,denstoredanske.dk
7950,"Den britiske forfatter og filosof Iris Murdoch døde, 79 år gammel. Hendes romaner udforsker eksistentielle problemer ud fra et moralfilosofisk udgangspunkt, fx Klokken og Et afhugget hoved.\nHandel begyndte på Nasdaq, verdens første elektroniske aktiemarked. Det har siden overhalet New York Stock Exchange målt på omsætning.\nUndervisningsminister Julius Bomholt udsendte en bekendtgørelse, der medførte, at det nye bogstav å fik pladsen sidst i alfabetet.\nDen amerikanske skuespiller Jack Lemm...",133,http://denstoredanske.dk/index.php?title=Rejser%2

In [141]:
all_records.iloc[10766]["text"] # 2888, 2988, 1211

"Billedvævning kom til Danmark i renæssancen. I 1577 kontaktede Frederik 2. Hans Knieper fra Antwerpen med tilbud om at lede et værksted i Helsingør, hvor der skulle fremstilles vævede tapeter til Kronborg; af disse 40 Kronborgtapeter med portrætter af danske konger er syv bevaret på Kronborg, syv på Nationalmuseet.\nI Danmark var billedvævning frem til slutningen af 1700-t. knyttet til konge og adel, mens det i Sverige og Norge vandt udbredelse i folkekunsten med vævecentre i Gudbrandsdalen, i Skåne og Dalarna.\nI årene omkring 1900 fik billedvævningen en renæssance i Danmark. Seks af Karel van Mander 2.s vævede tapeter til Frederiksborg Slot blev udført på ny 1903-28 under ledelse af bl.a. Christiane Konstantin-Hansen, datter af maleren Constantin Hansen, og Johanne Bindesbøll, datter af Gottlieb Bindesbøll; de deltog også 1920-26 i udførelsen af seks billedtæpper til det nyopførte Christiansborg Slot med motiver fra danske folkeviser, tegnet af Joakim Skovgaard. Også Københavns nye 

In [133]:
all_records[all_records['text'].str.contains('DTU', case=False, na=False)]


,text,minhash_cluster_size,url,id,domain
217,"Min søster og jeg har ofte samtaler om, hvem vores mor nu bedst kan lide. Altså for sjov. De udspringer som regel af, at en af os er kommet til at gøre noget mindre heldigt. Den anden er ikke sen til straks at råbe HA, NU KAN HUN HELT KLART BEDST LIDE MIG. Og så skumler den degraderede. Men vi griner altid af det, for vi har aldrig haft fornemmelsen af, at en af os er blevet forfordelt, hverken som børn eller voksne.\nI pinsen besøgte Allan, ungerne og jeg min mor. Allan var noget så prægtig...",1,http://marensblog.dk/,<urn:uuid:3ac62489-5588-45ee-a5e1-d9d80f14f538>,marensblog.dk
370,"Campus Service er en serviceenhed, der varetager planlægning, projektstyring af bygge- og anlægsprojekter, drift og vedligeholdelse af DTU’s bygninger, installationer, veje, pladser og parkområder mv. Vi servicerer studerende og ansatte ved DTU, samt en lang række eksterne kunder, myndigheder og øvrige interessenter.\nEfter universitetsfusionen i 2007 er DTU også geografisk et landsdækkende universitet med aktiviteter fordelt på 15 lokaliteter på Sjælland og i Jylland. Campus Service har fas...",1,http://www.dtu.dk/Om_DTU/Organisation/Koncernstab/Campus_Service.aspx,<urn:uuid:840dd01f-5ddd-43ec-bbfe-670f93abedda>,www.dtu.dk
429,"Kunstmuseer\nOplev de græske og romerske guder og helte på en rundtur i Glyptotekets skulptursale. Vi kommer også ind på de mere folkelige religioner. Få svar på, hvorfor man tog varsler af dyrs indvolde, og hvad en mysteriekult er.\nSe, hvordan lokale kulter langsomt smeltede sammen og dannede den klassiske gudeverden. For så senere at blive opbrudt i mysteriekulter og langsomt bane vejen for monoteismen. Hør om Zeus, Mitras og Kristus selv. Og hvordan, alt dette hænger sammen.\nGratis entr...",22,http://www.aok.dk/udstilling/rundvisning-guder-og-tro-i-antikken-1,<urn:uuid:0926d7ee-3a27-4945-a49c-8aa3932590a6>,www.aok.dk
501,"Ledige stillinger på DTU opslås altid på vores jobsite. Da stillinger i staten skal annonceres offentligt, modtager vi ikke uopfordrede ansøgninger.\nAnsøgninger til opslåede stillinger skal sendes online via et link, som findes nederst i alle jobbeskrivelser. Ansøgningsfristen fremgår altid af det enkelte stillingsopslag.\nNår du har sendt din ansøgning, vil du modtage en kvittering. Hvis du mod forventning ikke modtager en bekræftelse, skal du sende en e-mail til vores rekrutteringsafdeling",7,http://www.job.dtu.dk/Rekrutteringsproces.aspx,<urn:uuid:36f941e6-2cd1-4c60-9099-63e7027a8ed4>,www.job.dtu.dk
554,"Vidensdeling og samarbejde med eksterne partnere hjælper os til at sætte forskningsresultater i anvendelse. Derudover skaber det aktive samarbejde med eksterne forskere og virksomheder et innovativt forskningsmiljø på DTU Nanotech, og er med til at sikre at det omgivende samfund får gavn af det innovative forskerarbejde.\nDTU Nanotech koordinerer og deltager i en række større forskningsprojekter, som rækker udover instituttets egne mure.\nHer kan du læse om nogle af de projekter, som er drev...",1,http://www.nanotech.dtu.dk/Innovation/Research_Projects.aspx,<urn:uuid:d485a929-222d-43b3-8ded-9d9deebbbb99>,www.nanotech.dtu.dk
587,"Bjergkæden Pyrenæerne danner grænse mellem Spanien og Frankrig og har ord for at være et af de sidste vilde områder i Europa.\nForuden et antal cirkulære ruter, går der 3 klassiske vandreruter igennem Pyrenæerne fra kyst til kyst: Le Sentier de Piedmont, la Grande Randonnée 10 og la Haute Route Pyrénnéenne.\nDen højest beliggende rute Haute Route Pyrénéenne er uafmærket og holder sig hovedsageligt til toppe og bjergkamme i 1.500 til 2.000 meters højde, men med enkelte strækninger på over 3.0...",33,http://www.rejseavisen.dk/pyrenaeerne-12-dage-paa-haute-route.html,<urn:uuid:d6b28891-7543-45f2-b8bb-32470551c9ef>,www.rejseavisen.dk
652,"Program:\n14.00-14.10 Introduktion og velkomst ved kandidatstudieleder Jens E. Wilhjelm (DTU)\n14.10-14.30 Perspektiverne inden for Signal- og modelbaseret diagnostik ved Lektor Helge Bjarup Dis

In [66]:
labels = ["None", "Minimal", "Basic", "Good", "Excellent"]
sampel_id = 7185

save_text_label(all_records.iloc[sampel_id]["id"],all_records.iloc[sampel_id]["text"], labels[4])

In [67]:
pd.read_csv("fineweb2_data.csv")  # Load the saved CSV to check the saved labels

,text,label,id
0,"\n\n1. Befolkningsstatistik\nDette kapitel omhandler befolkningsstatistik og indledes med en status over antallet af udlændinge i Danmark, deres statsborgerskab og herkomst. Dernæst ses på udviklingen i antallet af udlændinge i Danmark samt vandringer foretaget af udlændige de seneste år. I de efterfølgende afsnit ses på opholdstiden for indvandrere, naturalisationer samt sammensætningen af gruppen af udlændinge med hensyn til køn og alder.\nKapitlet indeholder endvidere en beskrivelse af ud...",['Excellent'],<urn:uuid:f9985ba7-c634-4987-97df-9a3708b3a03e>
1,"Du kan bruge visningstilstande, når du vil læse dokumentet eller flytte rundt i det.\nHvis du vil læse dokumentet uden at anstrenge øjnene mere end højst nødvendigt og bruge værktøjer, der er specielt beregnet til læsning af dokumenter, skal du vælge læselayout (læselayout: En visning, der er beregnet til læsning af dokumenter på en computerskærm. Dokumentet får en størrelse, der passer til skærmen, de fleste værktøjslinjer er fjernet, men der er stadig kommandoer til navigering, kommenterin...",['Good'],<urn:uuid:1426ecd3-5445-436e-bcf7-73551e5f8199>
2,"\n\nDen indre barriere…\nPerspektiv Nr. 2-2000\nSkær ned på fedtet – spis flere kulhydrater – men hvilke?\nTatjana Hejgaard Vasilaras, M. Sc., Ph.D. stud. & Arne Astrup, professor dr.med., Forskningsinstitut for Human Ernæring.\nAnbefalingerne er klare: Vi skal skære ned på fedtet og erstatte det med fødevarer, der har et højere indhold af kulhydrat og protein.\nVæk med piskefløden, osten, de fede kød- og pålægsprodukter, chipsene og fedtstofferne. Ind med brød, pasta, ris, grøntsager og de ...",['Good'],<urn:uuid:13ca953b-a3e7-4a94-a18d-de949a67afc1>
3,"\n\nDe hører ligesom Cirrus og Cirrostratus til de høje skyer, hvilket vil sige, at de typisk befinder sig i tre til otte kilometers højde i polare områder, fem til 13 kilometers højde i tempererede områder og seks til 18 kilometer i tropiske områder.\nCirrocumulus kan dannes i klar luft og kan være resultat af omdannelsen af Cirrus- eller Cirrostratusskyerne. De kan også dannes som resultat af formindskelse af de elementer som en Altocumulusflage eller et lag består af.",['Good'],<urn:uuid:3c3bff4a-6cfa-432a-b5af-83d8c49852be>
4,"Rens din skærm billigst og bedst\nHvordan renser man mest effektivt sin computer- eller tv-skærm? Fotograf og forfatter til blandt andet Politikens store bog om digital fotografering Thomas Nykrog har undersøgt emnet.\nBomuld og lunkent vand\nHan har blandt andet talt med mange computerfirmaer, og det viser sig, at en skærm renses bedst med en bomuldsklud, der ikke fnuller og almindeligt lunkent vand. De firmaer som selv fabrikerer rensevæske til skærme, anbefaler selvfølgelig, at man bruger...",['Good'],<urn:uuid:a524534e-0c82-4bae-9aef-0070d994c46a>
5,"Pil løgene og hak dem fint. Skyl bladselleristænglerne og skær dem ud i mindre stykker. Steg løgene klare i smør i en gryde med bladselleri, tilsæt skyllede timianblade eller tørret timian, paprika og majskerner.\nHæld mælk i og kog suppen ca. 15 min. Tilsæt salt. Blend suppen, til den er helt jævn og glat (kom den evt. igennem en grov si).\nSkær kyllingekødet i små tern. Kom kyllingekødet i den kogende suppe, og skru ned, så suppen småkoger i ca. 10 min., til kyllingekødet er gennemkogt.\nT...",['Good'],<urn:uuid:71124d83-6bd3-41fe-b535-c75dd399260d>
6,"Tomaterne skoldes, flås og skæres i små tern (fjern også de fleste kerner). De hakkede tomater puttes i en gryde og bringes i kog sammen med bouillon, basilikum eller timian, laurbærbladet, salt, peber og chili. Suppen bringes i kog og småkoger ca. 15 min. Herefter fjernes basilikum/timian og laurbærbladet og suppen pureres med en stavblender eller i en blender/food-processor. Put Creme Fine og kødboller i den purerede suppe, varm den op og smag evt. til med krydderier.\nServeres med et par ...",['Good'],<urn:uuid:d51a1e4d-d900-45d2-97ec-19a11d7ce338>
7,"Varm ovnen op til 200o . Udrør gæren med vand. Tilsæt olie, salt og 2/3 af